In [1]:
from q_code import *
from copy import deepcopy
from experiments_utils import ResetExperimentID
import reset
Precision.PRECISION = MAX_PRECISION
Precision.update_threshold()

In [2]:
batch = "B20"
hardware_spec = HardwareSpec.JOHANNESBURG
noise_model = NoiseModel(hardware_spec, thermal_relaxation=False)

In [3]:
embedding_index = 0
embedding = {0: 17, 1: 15, 2: 16}

In [4]:
final_embedding = deepcopy(embedding)
final_embedding

{0: 17, 1: 15, 2: 16}

In [5]:
values = sorted(final_embedding.values())
state_vector_embedding = dict()
for (key, value) in final_embedding.items():
    state_vector_embedding[key] = get_index(value, values)

In [6]:
state_vector_embedding

{0: 2, 1: 0, 2: 1}

## Algorithms

In [7]:
def load_algorithm(experiment_id):
    horizon = 7
    experiment_obj = type(experiment_id)
    config_path = get_config_path(experiment_id, batch)
    config = load_config_file(config_path, experiment_obj)
    output_dir = os.path.join(get_project_path(), config["output_dir"])
    algorithms_path = os.path.join(output_dir, "algorithms")
    algorithm_path = os.path.join(algorithms_path, f"{hardware_spec.value}_{embedding_index}_{horizon}.json")
    if isinstance(experiment_id, ResetExperimentID):
        local_embedding = {0:2}
    else:
        local_embedding = {0:0, 1:1, 2:2}
    actions_to_instructions = get_actions_to_instructions(noise_model, local_embedding, experiment_id)
    print(algorithm_path)
    # load algorithm json
    f_algorithm = open(algorithm_path)
    algorithm = AlgorithmNode(serialized=json.load(f_algorithm), actions_to_instructions=actions_to_instructions)
    f_algorithm.close() 
    return algorithm

In [8]:
optimal_bf = load_algorithm(BitflipExperimentID.IPMA2)
optimal_pf = load_algorithm(PhaseflipExperimentID.IPMA)
optimal_reset = load_algorithm(ResetExperimentID.main)

/Users/stefaniemuroyalei/Documents/algorithm_synthesis/results/bitflip/ipma2/B20/algorithms/fake_johannesburg_0_7.json
/Users/stefaniemuroyalei/Documents/algorithm_synthesis/results/phaseflip/ipma/B20/algorithms/fake_johannesburg_0_7.json
/Users/stefaniemuroyalei/Documents/algorithm_synthesis/results/reset/main/B20/algorithms/fake_johannesburg_0_7.json


In [9]:
from experiments_utils import get_default_flip_algorithm
default_bf = get_default_flip_algorithm(noise_model, {0:0, 1:1, 2:2}, 7, BitflipExperimentID.IPMA2, bitflip.get_experiments_actions, target_qubit=None)
default_pf = get_default_flip_algorithm(noise_model, {0:0, 1:1, 2:2}, 7, PhaseflipExperimentID.IPMA, phaseflip.get_experiments_actions, target_qubit=None)
default_reset = get_default_flip_algorithm(noise_model, {0:2}, 7, ResetExperimentID.main, reset.get_experiments_actions, target_qubit=None)


## Computed Guarantees

In [10]:
get_guarantees(noise_model, batch, hardware_spec, embedding, BitflipExperimentID.IPMA2, 7, bitflip.get_experiments_actions, bitflip.get_hardware_embeddings)

embedding index 0
/Users/stefaniemuroyalei/Documents/algorithm_synthesis/results/bitflip/ipma2/B20/pomdps/fake_johannesburg_0.txt
/Users/stefaniemuroyalei/Documents/algorithm_synthesis/results/bitflip/ipma2/B20/temp_algorithm.json
********


(0.944, 0.897)

In [11]:
get_guarantees(noise_model, batch, hardware_spec, embedding, PhaseflipExperimentID.IPMA, 7, phaseflip.get_experiments_actions, phaseflip.get_hardware_embeddings)

embedding index 0
/Users/stefaniemuroyalei/Documents/algorithm_synthesis/results/phaseflip/ipma/B20/pomdps/fake_johannesburg_0.txt
/Users/stefaniemuroyalei/Documents/algorithm_synthesis/results/phaseflip/ipma/B20/temp_algorithm.json
********


(0.943, 0.896)

In [12]:
get_guarantees(noise_model, batch, hardware_spec, {0: embedding[2]}, ResetExperimentID.main, 7, reset.get_experiments_actions, reset.get_hardware_embeddings)



embedding index 0
/Users/stefaniemuroyalei/Documents/algorithm_synthesis/results/reset/main/B20/pomdps/fake_johannesburg_0.txt
/Users/stefaniemuroyalei/Documents/algorithm_synthesis/results/reset/main/B20/temp_algorithm.json
********


(0.977, 0.879)

## Target States

In [13]:
def is_target_qs(state_vector):        
    qs = None
    for (index, amp) in enumerate(state_vector):
        if not isclose(amp, 0.0, abs_tol=Precision.isclose_abstol):
            if qs is None:
                qs = QuantumState(index, amp, qubits_used=list(final_embedding.keys()))
            else:
                assert qs.get_amplitude(index) == 0.0
                qs.add_amplitude(index, amp) 
    remove_indices = [state_vector_embedding[x] for x in range(2, len(final_embedding.keys()))]
    rho = qs.multi_partial_trace(remove_indices=remove_indices)
    assert len(rho) == 4
    assert len(rho[0]) == 4
#     print(rho)
    for i in range(len(rho)):
        for j in range(len(rho)):
            if (i == 0 and j == 0) or (i == 0 and j == 3) or (i == 3 and j == 0) or (i == 3 and j == 3):
                if not isclose(rho[i][j], 0.5, abs_tol=Precision.isclose_abstol):
                    return False
            else:
                if not isclose(rho[i][j], 0.0, abs_tol=Precision.isclose_abstol):
                    return False
    return True

In [14]:
def my_code(qc, cbits):
    execute_algorithm(optimal_bf, qc, cbits=cbits)
    execute_algorithm(optimal_reset, qc, cbits=cbits)
    execute_algorithm(optimal_pf, qc, cbits=cbits)

In [15]:
def default_code(qc, cbits):
    execute_algorithm(default_bf, qc, cbits=cbits)
    execute_algorithm(default_reset, qc, cbits=cbits)
    execute_algorithm(default_pf, qc, cbits=cbits)

In [30]:
def ibm_simulate_algorithms(is_default) -> float:   
    num_qubits = len(final_embedding.keys())
    ibm_noise_model = get_ibm_noise_model(hardware_spec, thermal_relaxation=False)
    
    accuracy = 0
    
    total = 0
    
    for qubit in [0, 1]:
        for error in [Op.I, Op.X, Op.Z, Op.Y]:
            initial_layout = dict()
            qr = QuantumRegister(num_qubits)
            cr = ClassicalRegister(num_qubits)
            for (key, val) in final_embedding.items():
                initial_layout[qr[key]] = val
            qc = QuantumCircuit(qr, cr)

            # prepare Bell state
            qc.append(NoiselessH, [0])
            qc.append(NoiselessCX, [0, 1])

            # inject error
            if error in [Op.X, Op.Y]:
                qc.append(NoiselessX, [qubit])
            if error in [Op.Z, Op.Y]:
                qc.append(NoiselessX, [qubit])
                
            # run actual algorithm
            if is_default:
                default_code(qc, cbits=cr)
            else:
                my_code(qc, cbits=cr)

            qc.save_statevector('res', pershot=True)
            state_vs = ibm_simulate_circuit(qc, ibm_noise_model, initial_layout)
            total+= len(state_vs)
            for (index,state) in enumerate(state_vs):
                if is_target_qs(state):
                    accuracy += 1
    return accuracy / total

In [31]:
print("accuracy of synthesized algorithms: ")
ibm_simulate_algorithms(is_default=False)

accuracy of synthesized algorithms: 


0.8558349609375

In [33]:
print("accuracy of traditional algorithms: ")
ibm_simulate_algorithms(is_default=True)

accuracy of traditional algorithms: 


0.6474609375